In [3]:
import pandas as pd
from datetime import timedelta
import json
import warnings
import numpy as np
# Suppress all warnings
warnings.filterwarnings("ignore")

# Your code that generates warnings


In [23]:
Spo_dict = {
        "name": [
            "SPO 07.12 TO 15.12 (1)",
            "SPO 07.12 TO 15.12 (2)",
            "SPO 18.12 TO 31.01",
            "spo 01.03 to 07.03",
            "SPO 14.03 TO 31.03",
            "SPO 01.04 TO 15.04",
        ]
}

In [24]:
file_path = r"D:\vscoded\Credit_App\Credit_App\test files\jonathan.xlsx"
statment = pd.read_excel(file_path,sheet_name='statment')
con = pd.read_excel(file_path,sheet_name= "contract")

In [25]:
def invoice_optimizer(cell,invoice):
    begin_date = invoice['first date'].iloc[0]
    end_date = invoice['second date'].iloc[-1]
    
    arr = cell['Arrival']
    dep = cell['Departure'] - timedelta(1)
    code = cell['Rate code']

    if begin_date > arr:
        arr = begin_date
    if end_date < dep:
        dep = end_date
    
    date_range = con[(arr<=con["second date"]) & (dep>=con["first date"])]
    empty_df = date_range.empty
    
    days = 0 
    price = 0
    
    if not empty_df:
        date_range['first date'].iloc[0] = arr
        date_range['second date'].iloc[-1] = dep
        
        days = np.array((date_range['second date'] - date_range['first date']).dt.days)
        price = np.array(date_range[cell['Rate code']])
    
    return days, price, arr, dep, empty_df

In [26]:
def contract_removal(con, date1, date2,cell):
    
    arr = cell['Arrival']
    dep = cell['Departure'] - timedelta(1)
    code = cell['Rate code']
    
    con1 = con[(arr<=con["second date"]) & (date1>=con["first date"])]
    con2 = con[(date2<=con["second date"]) & (dep>=con["first date"])]
    
    con1['first date'].iloc[0] = arr
    con1['second date'].iloc[-1] = date1
    
    
    con2['first date'].iloc[0] = date2
    con2['second date'].iloc[-1] = dep
    
    result = pd.concat([con1, con2], axis=0)

    
    empty_df = result.empty
    days = 0 
    price = 0
    
    if not empty_df:
        days = np.array((result['second date'] - result['first date']).dt.days)
        price = np.array(result[cell['Rate code']])
    return result
    

In [27]:
def append_and_pad(original_array, new_array):
    if original_array.ndim == 1:
        original_array = [original_array]
    # Find the maximum length between the two arrays
    max_length = max(len(original_array[0]), len(new_array))
    
    # Pad both arrays with zeros to match the maximum length
    original_array_padded = np.pad(original_array, ((0, 0), (0, max_length - len(original_array[0]))), 'constant')
    new_array_padded = np.pad(new_array, (0, max_length - len(new_array)), 'constant')
    
    # Stack the two arrays vertically
    result = np.vstack((original_array_padded, new_array_padded))
    
    return result

In [28]:
Spo_list = list()
if len(Spo_dict["name"]) > 0:
    for spo_num in reversed(range(len(Spo_dict["name"]))):
        Spo_list.append(pd.read_excel(file_path,sheet_name=Spo_dict["name"][spo_num]))


In [ ]:
days = np.empty(0)
price = np.empty(0)
for guest in range(len(statment['Arrival'])):
    spo_days = np.empty(0)
    spo_price = np.zeros(0)
    contract = False
    if len(Spo_dict["name"]) > 0:
        for spo_num in range(len(Spo_list)):
            
            cell =  statment.iloc[guest,:]
            SPO = Spo_list[spo_num] 
            threshold = 2
            SPO = SPO.dropna(thresh=len(SPO.columns) - threshold + 1)
            invoice_result = invoice_optimizer(cell,SPO)
            
            spo_days = np.append(spo_days,invoice_result[0])
            spo_price = np.append(spo_price,invoice_result[1])
            
            date1 = invoice_result[2]
            date2 = invoice_result[3]
            
            empty_df = invoice_result[4]
            if isinstance(contract,bool):
                contract = con
        if not empty_df:
            contract = contract_removal(contract,date1,date2,cell)
    spo_days = np.append(spo_days,invoice_optimizer(cell,contract)[0])
    spo_price = np.append(spo_price,invoice_optimizer(cell,contract)[1])
    
    days = append_and_pad(days,spo_days)
    price = append_and_pad(price,spo_price)
    
        
days = np.delete(days, 0, axis=0)
price = np.delete(price, 0, axis=0)

import time

start_time = time.time()

Total = list()
for i in range(len(days)):
    sum = 0
    for j in range(len(days[i])):
        sum = sum + (days[i][j] * price[i][j])
    Total.append(sum)
    
end_time = time.time()

# Calculate the execution time
execution_time = end_time - start_time
execution_time

0.10453581809997559

In [29]:
start_time = time.time()


TOTAL = np.sum(np.multiply(days,price),axis=1)


end_time = time.time()

# Calculate the execution time
execution_time = end_time - start_time
execution_time

0.0010035037994384766

In [30]:
TOTAL

array([29826.816,  1890.   ,  1152.   , ..., 29826.816, 29826.816,
       29826.816])

In [270]:
np.dot(days[1],price[1].T)

945.0